# Automation Flow for Finance Dashboard

### Imports

In [253]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

### Consuption Process
- take in and parse the latest csv file that was put into the 'trasaction-files' folder. 
- add classification elements such as 'grocery', 'entertainment/going out', 'subscriptions', 'income', 'gas', 'bank transfer'
- add location based elements
- create new csv with these new elements
- take the new csv and append the main transaction file with the clean transactions, making sure that there are no duplicates in description and date. 

In [254]:
# Get the most recent transaction file that has been added to the raw-transaction-files folder. 

# # Function to find the most recent file in a directory
# def get_most_recent_file(directory, extension="csv"):
#     list_of_files = glob.glob(os.path.join(directory, f"*.{extension}"))
#     if not list_of_files:
#         return None
#     most_recent_file = max(list_of_files, key=os.path.getctime)
#     return most_recent_file

# Will use the function above eventually once the other functions work properly.
# For the time being the file path will be hard coded for testing purposes. 

file_path = 'raw-transaction-files/test-transactions.csv'

In [255]:
raw = pd.read_csv(file_path)
columns_to_drop = ['Check/Serial #']
raw = raw.drop(columns=columns_to_drop)
raw.head()

,Posted Date,Transaction Date,Transaction Type,Description,Amount,Daily Posted Balance
0,6/5/24,6/5/24,Deposit,MOBILE FROM ****8702 - TRUIST ONLINE TRANSFER,$500,NaN
1,6/5/24,6/5/24,POS,OC PARKS-PARKING 06-04 IRVINE CA2413 DEBIT CARD PURCHASE,($7),NaN
2,6/5/24,6/5/24,POS,NORDSTROM RACK # 27991 GR 06-04-24 LAGUNA NIGUEL CA 2413 DEBIT CARD PURCHASE-PIN,($129.17),$555.50
3,6/6/24,6/6/24,POS,BOARD AND BREW LAG 06-04 949-2071027 CA 2413 DEBIT CARD PURCHASE,($19.34),NaN
4,6/6/24,6/6/24,POS,LAS GOLONDRINAS 3 06-04 LAGUNA NIGUEL CA 2413 DEBIT CARD PURCHASE,($27.40),NaN


Need to find the unique patterns that exist so that I can map them out for future automation purposes. 

In [256]:
# creating an empty column for transaction classification

# 1: Create an empty column called classification and error to which we will be filling out based on description values
raw['Classification'] = None
raw['Error'] = None

# 2: Make Every Deposit state 'Deposit/Transfer' for the classification value
raw.loc[raw['Transaction Type'] == 'Deposit', 'Classification'] = 'Deposit/transfer'

# 3: Make a new df that only includes descriptions that do not include deposits transaction types. 
description_df = raw.loc[raw['Transaction Type'] != 'Deposit', ['Description']].reset_index(drop=True)


In [257]:
pd.set_option('display.max_colwidth', None)
description_df.head()

,Description
0,OC PARKS-PARKING 06-04 IRVINE CA2413 DEBIT CARD PURCHASE
1,NORDSTROM RACK # 27991 GR 06-04-24 LAGUNA NIGUEL CA 2413 DEBIT CARD PURCHASE-PIN
2,BOARD AND BREW LAG 06-04 949-2071027 CA 2413 DEBIT CARD PURCHASE
3,LAS GOLONDRINAS 3 06-04 LAGUNA NIGUEL CA 2413 DEBIT CARD PURCHASE
4,ENTERTAIN ST991 06-05 CARSON CA2413 DEBIT CARD PURCHASE


In [258]:
description_df.columns

Index(['Description'], dtype='object')

In [259]:
# Specific prefixes for Case 1
specific_strings = ['SQ *', 'TST *', 'AT *', 'LEVY@', 'TST*']

def extract_store(description):
    # Case 1: Check for specific prefixes
    for specific_string in specific_strings:
        if description.startswith(specific_string):
            after_string = description[len(specific_string):].lstrip()
            match = re.match(r'([A-Za-z0-9 &\'\"/\-\.]+)(?=[#0-9]|$)', after_string)
            if match:
                return match.group(0).strip()  # Return the store name
            return None

    # Case 2: Match the store name allowing letters, numbers, spaces, hyphens, and slashes until a special character appears
    match = re.match(r'([A-Za-z0-9\/\-\s]+?)(?=[#0-9]|$)', description)
    if match:
        return match.group(0).strip()  # Return the store name, stripping whitespace

    # Case 3: Match for .gov or .com
    if '.gov' in description or '.com' in description:
        match = re.search(r'([A-Za-z0-9 &\'\"/\-\.]+(?:\.gov|\.com))', description)
        if match:
            return match.group(0).strip()  # Return the store name
    
    # Case 4: Match for apostrophes
    match = re.search(r'([A-Za-z0-9 &\'\"/\-\.]+\'[A-Za-z0-9 &\'\"/\-\.]*)', description)
    if match:
        return match.group(0).strip()  # Return the store name with apostrophes

    return None  # Return None if no store name is found

# Apply the function to create a new 'store' column
description_df['store'] = description_df['Description'].apply(extract_store)

# Print the resulting DataFrame
description_df[['Description', 'store']].head(20)


,Description,store
0,OC PARKS-PARKING 06-04 IRVINE CA2413 DEBIT CARD PURCHASE,OC PARKS-PARKING
1,NORDSTROM RACK # 27991 GR 06-04-24 LAGUNA NIGUEL CA 2413 DEBIT CARD PURCHASE-PIN,NORDSTROM RACK
2,BOARD AND BREW LAG 06-04 949-2071027 CA 2413 DEBIT CARD PURCHASE,BOARD AND BREW LAG
3,LAS GOLONDRINAS 3 06-04 LAGUNA NIGUEL CA 2413 DEBIT CARD PURCHASE,LAS GOLONDRINAS
4,ENTERTAIN ST991 06-05 CARSON CA2413 DEBIT CARD PURCHASE,ENTERTAIN ST
5,SHEETZ 0456 0000 06-06 PRINCETONWV 2413 DEBIT CARD PURCHASE,SHEETZ
6,KROGER #7 1851 EARL CO 06-07-24MORGANTOWN 2413 DEBIT CARD PURCHASE-PIN,KROGER
7,MITCHS FUEL & FOOD 06-07 814-3953177 PA 2413 DEBIT CARD PURCHASE,None
8,ASHEBROOKE EXPR 06-09-24 MORGANTOWN 2413 DEBIT CARD PURCHASE-PIN,ASHEBROOKE EXPR
9,CHIPOTLE 4698 06-09 MORGANTOWN WV 2413 DEBIT CARD PURCHASE,CHIPOTLE


In [260]:
# finding why the none values are populating or how to update the code logic to better ingest the values. 

none_values_df = description_df[description_df['store'].isnull()]
len(none_values_df)





15

In [261]:
# Print the descriptions for these records
none_values_df[['Description', 'store']]

# might need to create a case three that has .com, .gov handeled
# case four might include values with an appostrophie

,Description,store
7,MITCHS FUEL & FOOD 06-07 814-3953177 PA 2413 DEBIT CARD PURCHASE,None
13,APPLE.COM/BILL 06-11 866-712-7753 CA 2413 DEBIT CARD PURCHASE,None
15,AMAZON PRIME*O36RD 06-12 888-802-3080 WA 2413 DEBIT CARD PURCHASE,None
25,APPLE.COM/BILL 06-14 866-712-7753 CA 2413 DEBIT CARD RECURRING PYMT,None
41,APPLE.COM/BILL 06-18 866-712-7753 CA 2413 DEBIT CARD PURCHASE,None
47,RECREATION.GOV 06-21 877-444-6777 NM 2413 DEBIT CARD PURCHASE,None
67,COOL BEANS BAR & G 06-27 KNOXVILLE TN 2413 DEBIT CARD PURCHASE,None
68,COOL BEANS BAR & G 06-28 KNOXVILLE TN 2413 DEBIT CARD PURCHASE,None
76,APPLE.COM/BILL 07-01 866-712-7753 CA 2413 DEBIT CARD PURCHASE,None
77,APPLE.COM/BILL 07-01 866-712-7753 CA 2413 DEBIT CARD RECURRING PYMT,None


In [200]:
# will use this to create a master list for mapping. 
# essentially trying to create a semi-autonomous vlookup however the look up is done in the background
# will have to work through error handling process eventually

unique_values = description_df['store'].unique()
unique_stores= pd.DataFrame(unique_values, columns=['unique_description'])
unique_stores

,unique_description
0,OC PARKS-PARKING
1,NORDSTROM RACK
2,BOARD AND BREW LAG
3,LAS GOLONDRINAS
4,ENTERTAIN ST
...,...
89,TARGET
90,WEB PMTS Key Home Sales a BKGQDBINTERNET PAYMENT
91,MEDITERRANEAN BAKE
92,IRON ROOSTER


In [120]:
unique_stores.to_csv('unique-stores-mapping-raw.csv')

In [18]:
# List of Patterns

patterns = [
    r'^(?P<business>.*?)\s#(?P<id>\d+)\s.*?(\d{2}-\d{2}-\d{2})\s(?P<location>.*?)\s.*?(DEBIT|CREDIT)',
    r'^(?P<business>.*?)\s.*?(\d{2}/\d{2}/\d{2})\s(?P<location>.*?)\s.*?(DEBIT|CREDIT)',
    r'^(?P<business>.*?)\s.*?(\d{4}-\d{2}-\d{2})\s(?P<location>.*?)\s.*?(DEBIT|CREDIT)',
]

In [19]:
# parse transaction file

def parse_transaction(transaction):
    for pattern in patterns:
        match = re.search(pattern, transaction)
        if match:
            return match.groupdict()
        return {"business": None, "id": None, "location": None}

In [20]:
# load and parse CSV file
def load_transactions(file_path):
    df = pd.read_csv(file_path)
    df[['business', 'id', 'location']] = df['Transaction'].apply(lambda x: pd.Series(parse_transaction(x)))
    df['Amount'] = df['Amount'].replace('[\$,]', '', regex=True).astype(float)
    df['Date'] = pd.to_datetime(df['Date'])
    return df

In [15]:
df

NameError: name 'df' is not defined